In [1]:
import numpy as np
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
from stable_baselines3.common.vec_env import DummyVecEnv
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
import quantstats as qs
import pandas as pd
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.vec_env import VecNormalize

#Quant Finance
from finta import TA
import quantstats as qs

device = 'mps'

In [2]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

In [3]:
df = pd.read_csv('../monty_python/monty_python/datasets/ethereum_daily_indicators.csv',low_memory=False)

renamer = {'date':'Date','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'}
df.rename(columns=renamer,inplace=True)
df.set_index('Date',inplace=True)

df.sort_index(inplace=True)
df.fillna(0, inplace=True)

df.head()

,Open,High,Low,Close,Volume,dividends,stock_splits,AD,ADD,ATAN,...,maxidx,MULT,OBV,SIN,SQRT,SUB,TAN,TANH,TYPPRICE,WCLPRICE
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09 00:00:00+00:00,308.644989,329.451996,307.056000,320.884003,893249984,0.0,0.0,2.097921e+08,636.507996,1.567680,...,0,101160.211957,893249984.0,0.427343,17.913235,22.395996,0.472678,1.0,319.130666,319.569000
2017-11-10 00:00:00+00:00,320.670990,324.717987,294.541992,299.252991,885985984,0.0,0.0,-3.995582e+08,619.259979,1.567455,...,0,95643.082808,7264000.0,-0.718532,17.298930,30.175995,1.033123,1.0,306.170990,304.441490
2017-11-11 00:00:00+00:00,298.585999,319.453003,298.191986,314.681000,842300992,0.0,0.0,6.463645e+07,617.644989,1.567619,...,0,95258.325404,849564992.0,0.498385,17.739250,21.261017,0.574867,1.0,310.775330,311.751747
2017-11-12 00:00:00+00:00,314.690002,319.153015,298.513000,307.907990,1613479936,0.0,0.0,-7.998539e+07,617.666016,1.567549,...,0,95271.324163,-763914944.0,0.031904,17.547307,20.640015,0.031920,1.0,308.524668,308.370499
2017-11-13 00:00:00+00:00,307.024994,328.415009,307.024994,316.716003,1041889984,0.0,0.0,-1.777932e+08,635.440002,1.567639,...,0,100831.615994,277975040.0,0.552078,17.796517,21.390015,-0.662129,1.0,317.385335,317.218002


In [4]:
#Add momentum, volatitlity, & distance to the df_frame
df['return'] = np.log(df['Close'] / df['Close'].shift(1))

df['momentum'] = df['return'].rolling(5).mean().shift(1)
df['volatility'] = df['return'].rolling(20).std().shift(1)
df['distance'] = (df['Close'] - df['Close'].rolling(50).mean()).shift(1)

In [5]:
df['RSI'] = TA.RSI(df,16)
df['SMA'] = TA.SMA(df, 20)
df['SMA_L'] = TA.SMA(df, 41)
df['OBV'] = TA.OBV(df)
df['VWAP'] = TA.VWAP(df)
df['EMA'] = TA.EMA(df)
df['ATR'] = TA.ATR(df)
df.fillna(0, inplace=True)

In [6]:
df = df.head(128).copy()

In [7]:
#Perform a simple linear regression direction prediction
lags = 5

cols = []
for lag in range(1, lags + 1):
  col = f'lag_{lag}'
  df[col] = df['Close'].shift(lag)
  cols.append(col)

df.dropna(inplace=True)

reg = np.linalg.lstsq(df[cols], df['Close'], rcond=None)[0]
df['Prediction'] = np.dot(df[cols], reg)

#Create a function to properly format data frame to be passed through environment
def signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:,'Close'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Open','High','Low','Close','Volume','return','momentum','volatility','distance','RSI','OBV','SMA','SMA_L','VWAP','EMA','ATR', 'Prediction']].to_numpy()[start:end]
    return prices, signal_features

#Replace default df process with custom function from above
class MyCustomEnv(StocksEnv):
    _process_df = signals
    
#Initialize an environment setting the window size and train df
window_size = 5
start_index = window_size
end_train_index = round(len(df)*0.70)
end_val_index = len(df)    

In [8]:
models_dir = "models/eth_daily_100"
logdir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


# env_build = lambda: Monitor(gym.make('stocks-v0', df=df, window_size=window_size, frame_bound=(start_index, end_index)))
# env = DummyVecEnv([env_build])

env2 = MyCustomEnv(df=df, window_size=window_size, frame_bound=(start_index, end_train_index))

#Create a Dummy Vector of our environment
env_maker = lambda: Monitor(env2)
env = DummyVecEnv([env_maker])

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)



model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=logdir, device='mps')

Using mps device


In [9]:
TIMESTEPS = 10000
iters = 1
for i in range(1000):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name="eth_daily_100")
    model.save(f"{models_dir}/{TIMESTEPS*i}")

Logging to logs/eth_daily_100_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 80       |
|    ep_rew_mean     | 185      |
| time/              |          |
|    fps             | 197      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 80          |
|    ep_rew_mean          | 196         |
| time/                   |             |
|    fps                  | 160         |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010565813 |
|    clip_fraction        | 0.0952      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.684      |
|    explained_variance   | -0.484      

In [ ]:
# #initialize our model and train
# policy_kwargs = dict(optimizer_class='RMSpropTFLike', optimizer_kwargs=dict(eps=1e-5))
# actor_critic = A2C('MlpPolicy', env, verbose=1) 
# actor_critic.learn(total_timesteps=500000)

In [ ]:
# model.save('stocks')
